Data preparing

In [1]:
import pandas as pd
import pickle

In [67]:
import twitter

CONSUMER_KEY = "eTG832aIi0pyU6EjfoTGge6gx"
CONSUMER_SECRET = "mjvQFaovxMuRJ0U5XpdYfPsmmqiaZFNsL7g4qkk942OdDip6hG"

ACCESS_TOKEN_KEY = "780018504787488769-KVrRy17YTgqcUzrM32ykuYipvptNKvB"
ACCESS_TOKEN_SECRET = "qtZmbBAiOrzaBxwGNdlP0bfVA3cz2AWmsv0kq1jfdIOEb"

api = twitter.Api(consumer_key=CONSUMER_KEY, 
                  consumer_secret=CONSUMER_SECRET, 
                  access_token_key=ACCESS_TOKEN_KEY, 
                  access_token_secret=ACCESS_TOKEN_SECRET,
                  sleep_on_rate_limit=True)

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test_submition.csv")
test.cls = None

In [47]:
def get_user_twts (u_id):
    kaggle_date = 1474243200
    timeline = []
    timeline = api.GetUserTimeline(user_id=u_id, count=200, max_id=777880466351357952 , include_rts=True, exclude_replies=False)
#    for i in xrange(20):
#        if len(timeline) == 0: return []
#        oldest_twt_id = timeline.pop().id
#        timeline += api.GetUserTimeline(user_id=u_id, max_id=oldest_twt_id, count=200, include_rts=False, exclude_replies=True) 
#        if timeline[-1].created_at_in_seconds < kaggle_date:
#            print "timeline get iterations:", i
#            break
#    oldest_twt_id = timeline.pop().id
#    timeline += api.GetUserTimeline(user_id=u_id, max_id=oldest_twt_id, count=200, include_rts=False, exclude_replies=True) 
#    if timeline[-1].created_at_in_seconds > kaggle_date: 
#        print "BAD_USER"
#        bad_users.add(u_id)
    return timeline


In [5]:
import unicodedata
import nltk
import re

wnl = nltk.stem.wordnet.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
tokenizer = nltk.RegexpTokenizer('[a-z]+')

def tokenize_twt(twt):
    text = twt.text
    #normalization
    text = ''.join(c for c in unicodedata.normalize('NFD', text.lower()) if not unicodedata.combining(c))
    #deleting URL's
    text = re.sub('https?://\S+', '', text)
    #deleting HTML symbols
    text = re.sub('&\w+;', '', text)    

    words = tokenizer.tokenize(text)
    tokens = [wnl.lemmatize(word) for word in words if word not in stopwords]
    return tokens

def get_user_vocab(twts):
    vocab = {}
    for twt in twts:
        tokens = tokenize_twt(twt)
        for token in tokens:
            if token in vocab:
                vocab[token] += 1
            else:
                vocab[token] = 1
    return vocab


In [33]:
user_tokens = {}

In [34]:
bad_users = set()

In [8]:
downloaded_users = set()

In [48]:
new_downloaded_users = set()

In [65]:
from tqdm import tqdm

In [ ]:
from twitter import TwitterError

user_ids = pd.concat([train, test])['uid']
for uid in tqdm(user_ids):
    if uid in downloaded_users: continue
    if len(user_tokens) >= 1000:
        print 'TRYING TO SAVE PICKLE'
        with open('user_tokens_'+str(len(downloaded_users))+'.pkl', 'wb') as f:
            pickle.dump(user_tokens, f, pickle.HIGHEST_PROTOCOL)
            print "pickle dumped", 'user_tokens_'+str(len(downloaded_users))+'.pkl'
            user_tokens = {}
    try:    
        twts = get_user_twts(uid)
    except TwitterError:
        twts = []
        print 'error'
        next
    vocab = get_user_vocab(twts)
    user_tokens[uid] = vocab
    downloaded_users.add(uid)
    new_downloaded_users.add(uid)
    print uid, len(twts), 'downloaded', len(vocab), 'words in vocab'
        
    

 64%|██████▍   | 7670/11947 [00:01<00:00, 4523.88it/s]

4065131953 199 downloaded 594 words in vocab
error
28992682 0 downloaded 0 words in vocab
1539677432 200 downloaded 1116 words in vocab
error
2581586548 0 downloaded 0 words in vocab
768294420 198 downloaded 788 words in vocab
1278705324 200 downloaded 973 words in vocab
21567649 0 downloaded 0 words in vocab
508080349 200 downloaded 1055 words in vocab
469344128 0 downloaded 0 words in vocab
162106726 198 downloaded 841 words in vocab
3053986571 200 downloaded 948 words in vocab
3269434189 200 downloaded 523 words in vocab
2788368130 161 downloaded 637 words in vocab
2652732326 197 downloaded 941 words in vocab
549308820 0 downloaded 0 words in vocab
3023290870 199 downloaded 904 words in vocab
3071603159 200 downloaded 769 words in vocab
263348660 199 downloaded 937 words in vocab
2376564780 200 downloaded 704 words in vocab
1516587925 200 downloaded 845 words in vocab
error
712671618 0 downloaded 0 words in vocab
error
731995571855101952 0 downloaded 0 words in vocab
775023583 193 d

In [60]:
reload (twitter)

<module 'twitter' from '/home/dima/anaconda2/lib/python2.7/site-packages/twitter/__init__.pyc'>

In [55]:
len(user_tokens)

536

In [56]:
len(downloaded_users)

5674

In [57]:
len(new_downloaded_users)

3393

In [38]:
len(bad_users)

0

In [31]:
import time
def save_downloaded_users_set():
    t = int(time.time())
    with open(str(t)+'_downloaded_users.pkl', 'wb') as f:
        pickle.dump(downloaded_users, f, pickle.HIGHEST_PROTOCOL)
        print "pickle dumped", str(t)+'_downloaded_users.pkl'
    return

def save_bad_users_set():
    t = int(time.time())
    with open(str(t)+'_bad_users.pkl', 'wb') as f:
        pickle.dump(bad_users, f, pickle.HIGHEST_PROTOCOL)
        print "pickle dumped", str(t)+'_bad_users.pkl'
    return

def save_user_tokens():
    t = int(time.time())
    with open(str(t)+'_user_tokens.pkl', 'wb') as f:
        pickle.dump(user_tokens, f, pickle.HIGHEST_PROTOCOL)
        print "pickle dumped", str(t)+'_user_tokens.pkl'
    return

In [106]:
save_downloaded_users_set()
save_bad_users_set()
save_user_tokens()

pickle dumped 1482080100_downloaded_users.pkl
pickle dumped 1482080101_bad_users.pkl
pickle dumped 1482080101_user_tokens.pkl


In [13]:
def load_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [18]:
print 'load test'

print 'downloaded', len(load_pickle('1482080100_downloaded_users.pkl'))
print 'bad', len(load_pickle('1482080101_bad_users.pkl'))
print 'tokens', len(load_pickle('1482080101_user_tokens.pkl'))

load test
downloaded 1629
bad 338
tokens 629


In [11]:
user_tokens = load_pickle

1482080100_downloaded_users.pkl  test.csv
1482080101_bad_users.pkl	 test_submition.csv
1482080101_user_tokens.pkl	 train.csv
Project. Data Preraring.ipynb	 user_tokens_1.pkl
